# 🚧 Entrenamiento YOLOv8 - Detección de Baches

Este notebook entrena un modelo YOLOv8 para detectar baches en la calle y lo optimiza para deployment en Android.

## 📋 Pasos:
1. Instalar dependencias
2. Descargar dataset de Roboflow
3. Entrenar modelo YOLOv8
4. Optimizar para móvil (FP16 y INT8)
5. Exportar a TFLite

## 1️⃣ Instalar Dependencias

Instalamos Ultralytics (YOLOv8) y Roboflow para descargar el dataset.

In [ ]:
!pip install ultralytics roboflow -q
print("✅ Dependencias instaladas!")

## 2️⃣ Importar Librerías

In [ ]:
import os
from ultralytics import YOLO
from roboflow import Roboflow
import torch

print(f"🔧 PyTorch version: {torch.__version__}")
print(f"🖥️  CUDA disponible: {torch.cuda.is_available()}")
print(f"💾 Dispositivo: {'cuda' if torch.cuda.is_available() else 'cpu'}")

## 3️⃣ Descargar Dataset desde Roboflow

Descargamos el dataset de baches desde Roboflow (versión 18).

In [ ]:
# Conectar a Roboflow
rf = Roboflow(api_key="1kyEcz6MV5H8ZK9UIIBP")
project = rf.workspace("yeeun-kim-fyvoj").project("pothole-vhmow")
version = project.version(18)

# Descargar dataset en formato YOLOv8
print("📥 Descargando dataset...")
dataset = version.download("yolov11")

print(f"✅ Dataset descargado en: {dataset.location}")
print(f"📂 Archivo data.yaml: {dataset.location}/data.yaml")

## 4️⃣ Cargar Modelo Pre-entrenado

Cargamos YOLOv8 Nano (el más ligero) como base.

In [ ]:
# Cargar modelo YOLOv8 nano (más rápido para móvil)
model = YOLO('yolov8n.pt')
print("✅ Modelo YOLOv8n cargado")

## 5️⃣ Entrenar el Modelo

Entrenamos con configuraciones optimizadas para detección de baches.

In [ ]:
# Entrenar modelo
results = model.train(
    data=f'{dataset.location}/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,              # early stopping
    device='cuda' if torch.cuda.is_available() else 'cpu',
    workers=8,
    project='pothole_detection',
    name='yolov8n_v1',
    exist_ok=True,
    
    # Optimizaciones
    optimizer='AdamW',        # mejor para datasets pequeños
    lr0=0.001,                # learning rate inicial
    lrf=0.01,                 # learning rate final
    momentum=0.937,
    weight_decay=0.0005,
    
    # Data Augmentation
    hsv_h=0.015,              # hue
    hsv_s=0.7,                # saturation
    hsv_v=0.4,                # value
    degrees=10.0,             # rotación
    translate=0.1,            # traslación
    scale=0.5,                # zoom
    flipud=0.0,               # no voltear verticalmente
    fliplr=0.5,               # sí voltear horizontalmente
    mosaic=1.0,               # mosaic augmentation
)

print("\n🎉 Entrenamiento completado!")
print(f"📂 Resultados guardados en: {results.save_dir}")

## 6️⃣ Validar el Modelo

Probamos el modelo en el conjunto de validación.

In [ ]:
# Validar modelo
metrics = model.val()

print(f"\n📊 Métricas del modelo:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")
print(f"  Precisión: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")

## 7️⃣ Exportar a TFLite con FP16 (Optimización Float16)

Exportamos el modelo a TensorFlow Lite con precisión FP16 para reducir tamaño.

In [ ]:
# Cargar mejor modelo entrenado
best_model = YOLO('pothole_detection/yolov8n_v1/weights/best.pt')

# Exportar a TFLite FP16
print("📱 Exportando a TFLite FP16...")
fp16_path = best_model.export(
    format='tflite',
    half=True,           # FP16 precision
    imgsz=640,
    int8=False
)

print(f"✅ Modelo FP16 exportado: {fp16_path}")

# Renombrar para identificar fácilmente
import shutil
shutil.copy(fp16_path, 'models/best_float16.tflite')
print(f"📂 Copiado a: models/best_float16.tflite")

## 8️⃣ Exportar a TFLite con INT8 (Cuantización Máxima)

Exportamos con cuantización INT8 para máxima optimización (más pequeño, más rápido).

In [ ]:
# Exportar a TFLite INT8
print("📱 Exportando a TFLite INT8...")
int8_path = best_model.export(
    format='tflite',
    half=False,
    imgsz=640,
    int8=True           # INT8 quantization
)

print(f"✅ Modelo INT8 exportado: {int8_path}")

# Copiar a carpeta models
shutil.copy(int8_path, 'models/best_int8.tflite')
print(f"📂 Copiado a: models/best_int8.tflite")

## 9️⃣ Comparar Tamaños de Modelos

Veamos la diferencia de tamaño entre los modelos.

In [ ]:
import os

def get_file_size(path):
    size_bytes = os.path.getsize(path)
    size_mb = size_bytes / (1024 * 1024)
    return size_mb

print("\n📊 Comparación de tamaños:")
print(f"  Original PT:  {get_file_size('pothole_detection/yolov8n_v1/weights/best.pt'):.2f} MB")
print(f"  FP16 TFLite:  {get_file_size('models/best_float16.tflite'):.2f} MB")
print(f"  INT8 TFLite:  {get_file_size('models/best_int8.tflite'):.2f} MB")

print("\n💡 Recomendación:")
print("  - FP16: Mejor balance entre velocidad y precisión")
print("  - INT8: Más rápido, ligeramente menos preciso")

## 🔟 Probar Modelo en Imagen de Prueba

Probamos el modelo en una imagen para verificar que funciona.

In [ ]:
# Probar detección en imagen de validación
results = best_model.predict(
    source=f'{dataset.location}/valid/images',
    conf=0.5,           # confianza mínima
    save=True,          # guardar imágenes con detecciones
    project='predictions',
    name='test'
)

print(f"\n✅ Predicciones guardadas en: predictions/test")
print(f"📸 Revisa las imágenes para ver las detecciones")

## ✅ Resumen Final

### 📦 Archivos Generados:

1. **Modelo original**: `pothole_detection/yolov8n_v1/weights/best.pt`
2. **TFLite FP16**: `models/best_float16.tflite` ← **Recomendado para Android**
3. **TFLite INT8**: `models/best_int8.tflite` ← Más rápido, ligeramente menos preciso

### 📱 Siguiente Paso:

Copia los archivos `.tflite` a tu app Android:
```bash
cp models/best_float16.tflite PothholeDetector/app/src/main/assets/
```

### 🚀 Para usar en la app:

El modelo ya está integrado en `PotholeDetector.kt` y debería funcionar automáticamente.

---

**¡Listo para hackathon! 🎉**